In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/uidai-data/api_data_aadhar_enrolment_0_500000.csv
/kaggle/input/uidai-data/api_data_aadhar_enrolment_500000_1000000.csv
/kaggle/input/uidai-data/api_data_aadhar_enrolment_1000000_1006029.csv


In [2]:
import pandas as pd

files = [
    "/kaggle/input/uidai-data/api_data_aadhar_enrolment_0_500000.csv",
    "/kaggle/input/uidai-data/api_data_aadhar_enrolment_500000_1000000.csv",
    "/kaggle/input/uidai-data/api_data_aadhar_enrolment_1000000_1006029.csv"
]

df = pd.concat(
    (pd.read_csv(f) for f in files),
    ignore_index=True
)


In [3]:
df.shape

(1006029, 7)

In [4]:
df.to_csv("uidai_enrolment_full.csv", index=False)


In [5]:
df1 = df.copy()

In [6]:
df.isnull().sum()

date              0
state             0
district          0
pincode           0
age_0_5           0
age_5_17          0
age_18_greater    0
dtype: int64

In [7]:
df[df.duplicated()].head()

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater
360605,13-10-2025,Punjab,Jalandhar,144041,2,1,0
360606,13-10-2025,Punjab,Jalandhar,144101,1,0,0
360607,13-10-2025,Punjab,Jalandhar,144102,2,0,0
360608,13-10-2025,Punjab,Jalandhar,144418,1,0,0
360609,13-10-2025,Punjab,Jalandhar,144419,1,0,0


In [8]:
df=df.drop_duplicates()

In [9]:
df.duplicated().sum()

np.int64(0)

In [10]:
df.isna().sum()

date              0
state             0
district          0
pincode           0
age_0_5           0
age_5_17          0
age_18_greater    0
dtype: int64

In [11]:

df = df[~((df['state'] == 100000) &
          (df['district'] == 100000) &
          (df['pincode'] == 100000))]


In [12]:
df['state']

0              Meghalaya
1              Karnataka
2          Uttar Pradesh
3          Uttar Pradesh
4              Karnataka
               ...      
1006024      West Bengal
1006025      West Bengal
1006026      West Bengal
1006027      West Bengal
1006028      West Bengal
Name: state, Length: 983072, dtype: object

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 983072 entries, 0 to 1006028
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   date            983072 non-null  object
 1   state           983072 non-null  object
 2   district        983072 non-null  object
 3   pincode         983072 non-null  int64 
 4   age_0_5         983072 non-null  int64 
 5   age_5_17        983072 non-null  int64 
 6   age_18_greater  983072 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 60.0+ MB


In [14]:
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y', errors='coerce')


In [15]:
df = df[df['state'] != '100000']


In [16]:
df['state'] = df['state'].str.lower().str.strip()

In [17]:
state_map = {
    'west bengal': 'west bengal',
    'west bangal': 'west bengal',
    'westbengal': 'west bengal',
    'west  bengal': 'west bengal',
    'odisha': 'odisha',
    'orissa': 'odisha',
    'andhra pradesh': 'andhra pradesh',
    'andhra pradesh': 'andhra pradesh',  # ensure spelling consistent
    'jammu & kashmir': 'jammu and kashmir',
    'jammu and kashmir': 'jammu and kashmir',
    'jammu and kashmir': 'jammu and kashmir',
    'dadra & nagar haveli': 'dadra and nagar haveli',
    'dadra and nagar haveli': 'dadra and nagar haveli',
    'daman & diu': 'daman and diu',
    'daman and diu': 'daman and diu',
    'pondicherry': 'puducherry',
    'andaman & nicobar islands': 'andaman and nicobar islands',
    'andaman and nicobar islands': 'andaman and nicobar islands',
    'westbengal': 'west bengal'
    # add more mappings as needed
}


In [18]:
df['state'] = df['state'].replace(state_map)


In [19]:
df['state'].unique()


array(['meghalaya', 'karnataka', 'uttar pradesh', 'bihar', 'maharashtra',
       'haryana', 'rajasthan', 'punjab', 'delhi', 'madhya pradesh',
       'west bengal', 'assam', 'uttarakhand', 'gujarat', 'andhra pradesh',
       'tamil nadu', 'chhattisgarh', 'jharkhand', 'nagaland', 'manipur',
       'telangana', 'tripura', 'mizoram', 'jammu and kashmir',
       'chandigarh', 'sikkim', 'odisha', 'kerala',
       'the dadra and nagar haveli and daman and diu',
       'arunachal pradesh', 'himachal pradesh', 'goa',
       'dadra and nagar haveli and daman and diu', 'ladakh',
       'andaman and nicobar islands', 'puducherry', 'lakshadweep',
       'dadra and nagar haveli', 'daman and diu'], dtype=object)

In [20]:
# All variations mapped to one canonical UT
dadra_map = {
    'dadra & nagar haveli': 'dadra and nagar haveli and daman and diu',
    'daman & diu': 'dadra and nagar haveli and daman and diu',
    'dadra and nagar haveli': 'dadra and nagar haveli and daman and diu',
    'daman and diu': 'dadra and nagar haveli and daman and diu',
    'dadra and nagar haveli and daman and diu': 'dadra and nagar haveli and daman and diu'
}

# Apply to your state column
df['state'] = df['state'].replace(dadra_map)


In [21]:
# Remove leading "the " from state names
df['state'] = df['state'].str.lower().str.strip().str.replace(r'^the\s+', '', regex=True)

# Now apply your canonical mapping
state_map = {
    'dadra and nagar haveli and daman and diu': 'dadra and nagar haveli and daman and diu',
    # add other mappings if needed
}

df['state'] = df['state'].replace(state_map)


In [22]:
df['district'].nunique()

984

In [23]:
# Remove asterisks and extra spaces
df['district'] = df['district'].str.replace(r'[*]', '', regex=True).str.strip()


In [24]:
import plotly.express as px

state_counts = df['state'].value_counts().reset_index()
state_counts.columns = ['state', 'count']

fig = px.bar(
    state_counts,
    x='state',
    y='count',
    title='State Count Distribution',
    hover_data={'state': True, 'count': True}
)

fig.update_layout(
    xaxis_title='State',
    yaxis_title='State Count',
    xaxis_tickangle=-45
)

fig.show()


# UP has highest enrollment in 2025

In [25]:
print(df['date'].min())
print(df['date'].max())

2025-03-02 00:00:00
2025-12-31 00:00:00


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 983051 entries, 0 to 1006028
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            983051 non-null  datetime64[ns]
 1   state           983051 non-null  object        
 2   district        983051 non-null  object        
 3   pincode         983051 non-null  int64         
 4   age_0_5         983051 non-null  int64         
 5   age_5_17        983051 non-null  int64         
 6   age_18_greater  983051 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 60.0+ MB


In [27]:
df.describe()

,date,pincode,age_0_5,age_5_17,age_18_greater
count,983051,983051.000000,983051.000000,983051.000000,983051.000000
mean,2025-10-23 05:35:28.046255872,518478.258356,3.534292,1.720061,0.169115
min,2025-03-02 00:00:00,110001.000000,0.000000,0.000000,0.000000
25%,2025-09-18 00:00:00,363630.000000,1.000000,0.000000,0.000000
50%,2025-10-26 00:00:00,517403.000000,2.000000,0.000000,0.000000
75%,2025-11-15 00:00:00,700093.000000,3.000000,1.000000,0.000000
max,2025-12-31 00:00:00,855456.000000,2688.000000,1812.000000,855.000000
std,NaN,205485.496222,17.726430,14.529913,3.251934


In [28]:
df[df['age_0_5']!=0].count()

date              870293
state             870293
district          870293
pincode           870293
age_0_5           870293
age_5_17          870293
age_18_greater    870293
dtype: int64

In [29]:
# Age-wise total enrollment
total_age_0_5 = df['age_0_5'].sum()
total_age_5_17 = df['age_5_17'].sum()
total_age_18_greater = df['age_18_greater'].sum()

print(f"Total enrollment age 0-5: {total_age_0_5}")
print(f"Total enrollment age 5-17: {total_age_5_17}")
print(f"Total enrollment age 18+: {total_age_18_greater}")


Total enrollment age 0-5: 3474389
Total enrollment age 5-17: 1690908
Total enrollment age 18+: 166249


# Age wise count gives us the info that as the age is less the enrolment number is more its due to new enrollment

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 983051 entries, 0 to 1006028
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            983051 non-null  datetime64[ns]
 1   state           983051 non-null  object        
 2   district        983051 non-null  object        
 3   pincode         983051 non-null  int64         
 4   age_0_5         983051 non-null  int64         
 5   age_5_17        983051 non-null  int64         
 6   age_18_greater  983051 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 60.0+ MB


In [31]:
df['pincode'].value_counts().head(30)

pincode
500055    269
500018    263
500005    242
110053    216
431001    214
743329    212
244102    206
831002    200
713143    199
110032    194
500087    193
831012    190
110093    190
500049    190
853204    190
500072    189
700135    186
450661    184
400078    181
497001    181
735211    180
396230    178
515865    177
736121    176
600100    176
110094    175
509129    174
400097    174
500008    174
782446    172
Name: count, dtype: int64

In [32]:
df

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater
0,2025-03-02,meghalaya,East Khasi Hills,793121,11,61,37
1,2025-03-09,karnataka,Bengaluru Urban,560043,14,33,39
2,2025-03-09,uttar pradesh,Kanpur Nagar,208001,29,82,12
3,2025-03-09,uttar pradesh,Aligarh,202133,62,29,15
4,2025-03-09,karnataka,Bengaluru Urban,560016,14,16,21
...,...,...,...,...,...,...,...
1006024,2025-12-31,west bengal,West Midnapore,721149,2,0,0
1006025,2025-12-31,west bengal,West Midnapore,721150,2,2,0
1006026,2025-12-31,west bengal,West Midnapore,721305,0,1,0
1006027,2025-12-31,west bengal,West Midnapore,721504,1,0,0


In [33]:
df['total_enroll'] = df['age_0_5'] + df['age_5_17'] + df['age_18_greater']

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 983051 entries, 0 to 1006028
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date            983051 non-null  datetime64[ns]
 1   state           983051 non-null  object        
 2   district        983051 non-null  object        
 3   pincode         983051 non-null  int64         
 4   age_0_5         983051 non-null  int64         
 5   age_5_17        983051 non-null  int64         
 6   age_18_greater  983051 non-null  int64         
 7   total_enroll    983051 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(2)
memory usage: 67.5+ MB


In [35]:
df.describe()

,date,pincode,age_0_5,age_5_17,age_18_greater,total_enroll
count,983051,983051.000000,983051.000000,983051.000000,983051.000000,983051.000000
mean,2025-10-23 05:35:28.046255872,518478.258356,3.534292,1.720061,0.169115,5.423468
min,2025-03-02 00:00:00,110001.000000,0.000000,0.000000,0.000000,1.000000
25%,2025-09-18 00:00:00,363630.000000,1.000000,0.000000,0.000000,1.000000
50%,2025-10-26 00:00:00,517403.000000,2.000000,0.000000,0.000000,2.000000
75%,2025-11-15 00:00:00,700093.000000,3.000000,1.000000,0.000000,5.000000
max,2025-12-31 00:00:00,855456.000000,2688.000000,1812.000000,855.000000,3965.000000
std,NaN,205485.496222,17.726430,14.529913,3.251934,31.932495


In [36]:
df['state'].unique()

array(['meghalaya', 'karnataka', 'uttar pradesh', 'bihar', 'maharashtra',
       'haryana', 'rajasthan', 'punjab', 'delhi', 'madhya pradesh',
       'west bengal', 'assam', 'uttarakhand', 'gujarat', 'andhra pradesh',
       'tamil nadu', 'chhattisgarh', 'jharkhand', 'nagaland', 'manipur',
       'telangana', 'tripura', 'mizoram', 'jammu and kashmir',
       'chandigarh', 'sikkim', 'odisha', 'kerala',
       'dadra and nagar haveli and daman and diu', 'arunachal pradesh',
       'himachal pradesh', 'goa', 'ladakh', 'andaman and nicobar islands',
       'puducherry', 'lakshadweep'], dtype=object)

In [37]:
# Create 5 binary features for regions (NEWSC)
df['is_north'] = (df['state'].str.lower().str.strip().isin([
    'delhi', 'chandigarh', 'jammu and kashmir', 'ladakh',
    'punjab', 'haryana', 'himachal pradesh', 'uttarakhand',
    'uttar pradesh', 'rajasthan'
])).astype(int)

df['is_east'] = (df['state'].str.lower().str.strip().isin([
    'west bengal', 'odisha', 'bihar', 'jharkhand', 'sikkim',
    'assam', 'meghalaya', 'arunachal pradesh', 'nagaland',
    'manipur', 'mizoram', 'tripura'
])).astype(int)

df['is_west'] = (df['state'].str.lower().str.strip().isin([
    'gujarat', 'maharashtra', 'goa',
    'dadra and nagar haveli and daman and diu'
])).astype(int)

df['is_south'] = (df['state'].str.lower().str.strip().isin([
    'andhra pradesh', 'telangana', 'karnataka', 'kerala',
    'tamil nadu', 'puducherry', 'lakshadweep',
    'andaman and nicobar islands'
])).astype(int)

df['is_central'] = (df['state'].str.lower().str.strip().isin([
    'madhya pradesh', 'chhattisgarh'
])).astype(int)

# Create 1 binary feature for UT
df['is_ut'] = (df['state'].str.lower().str.strip().isin([
    'delhi', 'chandigarh', 'jammu and kashmir', 'ladakh',
    'dadra and nagar haveli and daman and diu',
    'puducherry', 'lakshadweep', 'andaman and nicobar islands'
])).astype(int)

In [38]:
df

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater,total_enroll,is_north,is_east,is_west,is_south,is_central,is_ut
0,2025-03-02,meghalaya,East Khasi Hills,793121,11,61,37,109,0,1,0,0,0,0
1,2025-03-09,karnataka,Bengaluru Urban,560043,14,33,39,86,0,0,0,1,0,0
2,2025-03-09,uttar pradesh,Kanpur Nagar,208001,29,82,12,123,1,0,0,0,0,0
3,2025-03-09,uttar pradesh,Aligarh,202133,62,29,15,106,1,0,0,0,0,0
4,2025-03-09,karnataka,Bengaluru Urban,560016,14,16,21,51,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006024,2025-12-31,west bengal,West Midnapore,721149,2,0,0,2,0,1,0,0,0,0
1006025,2025-12-31,west bengal,West Midnapore,721150,2,2,0,4,0,1,0,0,0,0
1006026,2025-12-31,west bengal,West Midnapore,721305,0,1,0,1,0,1,0,0,0,0
1006027,2025-12-31,west bengal,West Midnapore,721504,1,0,0,1,0,1,0,0,0,0
